In [1]:
%matplotlib inline
# !pip install stargazer
# !pip install scipy
# !pip install plotly_express
# !pip install pandas-profiling
# !pip install linearmodels
# !pip install sklearn

import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
# import sklearn

from linearmodels.panel import compare
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api


In [2]:
path="/Users/mengxi/Documents/GitHub/research_module/data/"
data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")

# path="/Users/effie/Desktop/term3/research_module/data/"
# data=pd.read_stata(path+"county_clean/total_and_by_industry_data_county.dta")

data=data.drop(columns=['index'])

data=data.rename(columns={"ln_gdp": "gdp_growth_100"})

data




,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,toas_ratio,emp_agr_weight,emp_man_weight,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,1.0,0.002413,0.213166,0.784421,0.000634,0.324826,0.674540,0.002775,0.223713,0.773512
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,1.0,0.004872,0.422414,0.572714,0.001776,0.505710,0.492514,0.006840,0.446232,0.546928
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,1.0,0.006507,0.376005,0.617488,0.002375,0.408458,0.589167,0.008329,0.397011,0.594661
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,1.0,0.013096,0.400742,0.586162,0.006225,0.432973,0.560802,0.019670,0.424832,0.555498
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,1.0,0.014722,0.341771,0.643507,0.005007,0.376126,0.618868,0.018984,0.361308,0.619708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,NaN,0.044653,0.428515,0.526831,0.031515,0.474068,0.494417,NaN,NaN,NaN
3605,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.0,0.037358,0.373862,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN
3606,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,1.0,0.028015,0.338902,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN
3607,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.0,0.002846,0.296681,0.700473,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data["No_ratio"].describe()

count    2709.000000
mean        0.286266
std         0.338948
min         0.000000
25%         0.000000
50%         0.166667
75%         0.500000
max         1.000000
Name: No_ratio, dtype: float64

In [4]:
data["toas_ratio"].describe()

count    2709.000000
mean        0.627292
std         0.475216
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.146924
Name: toas_ratio, dtype: float64

In [5]:
query_data = data.query('No_ratio>0.286')
data.loc[:,'county_affected_no'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('toas_ratio>0.627')
data.loc[:,'county_affected_toas'] = data.index.map(lambda x : 1 if x in query_data.index else 0)

query_data = data.query('year>2008')
data.loc[:,'year_post'] = data.index.map(lambda x : 1 if x in query_data.index else 0)
data

,year,Gebietseinheit,EU_Code,Regional_key,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,...,emp_ser_weight,gva_agr_weight,gva_man_weight,gva_ser_weight,vow_agr_weight,vow_man_weight,vow_ser_weight,county_affected_no,county_affected_toas,year_post
0,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,08111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,...,0.784421,0.000634,0.324826,0.674540,0.002775,0.223713,0.773512,1,1,0
1,2004,"Böblingen, Landkreis",DE112,08115,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,...,0.572714,0.001776,0.505710,0.492514,0.006840,0.446232,0.546928,0,1,0
2,2004,"Esslingen, Landkreis",DE113,08116,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,...,0.617488,0.002375,0.408458,0.589167,0.008329,0.397011,0.594661,0,1,0
3,2004,"Göppingen, Landkreis",DE114,08117,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,...,0.586162,0.006225,0.432973,0.560802,0.019670,0.424832,0.555498,1,1,0
4,2004,"Ludwigsburg, Landkreis",DE115,08118,BW,504.459991,0.341932,15851.325,31422.0,67176.0,...,0.643507,0.005007,0.376126,0.618868,0.018984,0.361308,0.619708,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,2012,Saale-Orla-Kreis,DEG0K,16075,TH,84.847000,5.540099,1955.848,23051.0,48013.0,...,0.526831,0.031515,0.474068,0.494417,NaN,NaN,NaN,0,0,1
3605,2012,"Greiz, Landkreis",DEG0L,16076,TH,103.880997,2.040082,1854.354,17851.0,46776.0,...,0.588780,0.031252,0.365997,0.602751,NaN,NaN,NaN,0,0,1
3606,2012,"Altenburger Land, Landkreis",DEG0M,16077,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,...,0.633083,0.023566,0.340479,0.635954,NaN,NaN,NaN,1,1,1
3607,2012,"Eisenach, Kreisfreie Stadt",DEG0N,16056,TH,NaN,NaN,NaN,NaN,NaN,...,0.700473,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1


In [6]:
data.columns

Index(['year', 'Gebietseinheit', 'EU_Code', 'Regional_key', 'Land', 'resident',
       'gdp_growth_100', 'gdp_level', 'gdp_per_inhabitant', 'gdp_per_employee',
       'gdp_per_hour_per_employed', 'ln_gva', 'gva_level', 'gva_agr',
       'gva_man', 'gva_ser', 'gva_ser_fin', 'gva_ser_trade', 'gva_ser_public',
       'employee', 'emp_agr', 'emp_man', 'emp_ser', 'emp_ser_fin',
       'emp_ser_trade', 'emp_ser_public', 'volume_of_work', 'vow_agr',
       'vow_man', 'vow_ser', 'vow_ser_fin', 'vow_ser_trade', 'vow_ser_public',
       'fixed_agr', 'fixed_man', 'fixed_ser', 'fixed_ser_trade',
       'fixed_ser_fin', 'fixed_ser_public', 'fixed_total', 'firms_no',
       'firms_aff_no', 'No_ratio', 'toas_sum_county', 'toas_aff_sum_county',
       'toas_ratio', 'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight',
       'gva_agr_weight', 'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight',
       'vow_man_weight', 'vow_ser_weight', 'county_affected_no',
       'county_affected_toas', 'year_post'

In [7]:


data["Land"].value_counts()

BY    864
NW    477
NI    405
BW    396
RP    324
HE    234
TH    207
BB    162
SH    135
ST    126
SN    117
MV     72
SL     54
HB     18
BE      9
HH      9
Name: Land, dtype: int64

In [8]:
data["affected_after_no"] = data.county_affected_no*data.year_post
data["affected_after_toas"] = data.county_affected_toas*data.year_post
data["ln_res"]=np.log(data["resident"]*1000)

data["ln_gdp_level"] = np.log(data.gdp_level*1000000)
data["ln_gdp_per_inhabitant"] = np.log(data.gdp_per_inhabitant)
data["ln_gdp_per_employee"] = np.log(data.gdp_per_employee)
data["ln_gdp_per_hour_per_employed"] = np.log(data.gdp_per_hour_per_employed)

data["ln_gva_level"] = np.log(data.gva_level*1000000)
data["ln_gva_agr"] = np.log(data.gva_agr*1000000)
data["ln_gva_man"] = np.log(data.gva_man*1000000)
data["ln_gva_ser"] = np.log(data.gva_ser*1000000)
data["ln_gva_ser_fin"] = np.log(data.gva_ser_fin*1000000)
data["ln_gva_ser_trade"] = np.log(data.gva_ser_trade*1000000)
data["ln_gva_ser_public"] = np.log(data.gva_ser_public*1000000)

data["ln_emp"]=np.log(data["employee"]*1000)

data["ln_emp_ser_fin"] = np.log(data.emp_ser_fin*1000)
data["ln_emp_ser_trade"] = np.log(data.emp_ser_trade*1000)
data["ln_emp_ser_public"] = np.log(data.emp_ser_public*1000)
data["ln_emp_agr"]=np.log(data["emp_agr"]*1000)
data["ln_emp_man"]=np.log(data["emp_man"]*1000)
data["ln_emp_ser"]=np.log(data["emp_ser"]*1000)

data["ln_vow"]=np.log(data["volume_of_work"]*1000000)

data["ln_vow_ser_fin"] = np.log(data.vow_ser_fin*1000000)
data["ln_vow_ser_trade"] = np.log(data.vow_ser_trade*1000000)
data["ln_vow_ser_public"] = np.log(data.vow_ser_public*1000000)
data["ln_vow_agr"]=np.log(data["vow_agr"]*1000000)
data["ln_vow_man"]=np.log(data["vow_man"]*1000000)
data["ln_vow_ser"]=np.log(data["vow_ser"]*1000000)

data["ln_fixed_total"] = np.log(data.fixed_total*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_fin"] = np.log(data.fixed_ser_fin*1000000)
data["ln_fixed_ser_trade"] = np.log(data.fixed_ser_trade*1000000)
data["ln_fixed_ser_public"] = np.log(data.fixed_ser_public*1000000)
data["ln_fixed_agr"]=np.log(data["fixed_agr"]*1000000)
data["ln_fixed_man"]=np.log(data["fixed_man"]*1000000)
data["ln_fixed_ser"]=np.log(data["fixed_ser"]*1000000)

data["gdp_growth"]=data["gdp_growth_100"]/100

df_fe = data.set_index(['Regional_key','year'])
df_fe

,,Gebietseinheit,EU_Code,Land,resident,gdp_growth_100,gdp_level,gdp_per_inhabitant,gdp_per_employee,gdp_per_hour_per_employed,ln_gva,...,ln_vow_man,ln_vow_ser,ln_fixed_total,ln_fixed_ser_fin,ln_fixed_ser_trade,ln_fixed_ser_public,ln_fixed_agr,ln_fixed_man,ln_fixed_ser,gdp_growth
Regional_key,year,,,,,,,,,,,,,,,,,,,,,
08111,2004,"Stuttgart, Landeshauptstadt, Stadtkreis",DE111,BW,573.955017,-1.059015,40680.228,70877.0,85811.0,58.340000,-0.712461,...,18.865405,20.105983,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018,-0.010590
08115,2004,"Böblingen, Landkreis",DE112,BW,366.351990,-3.269339,14478.332,39520.0,67825.0,47.110001,-2.930522,...,18.736418,18.939897,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018,-0.032693
08116,2004,"Esslingen, Landkreis",DE113,BW,503.649994,-0.074150,15205.272,30190.0,61149.0,42.700001,0.275855,...,18.766969,19.170997,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018,-0.000741
08117,2004,"Göppingen, Landkreis",DE114,BW,255.177994,-0.930964,6158.588,24134.0,54533.0,38.419998,-0.583964,...,18.036458,18.304630,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018,-0.009310
08118,2004,"Ludwigsburg, Landkreis",DE115,BW,504.459991,0.341932,15851.325,31422.0,67176.0,47.139999,0.693390,...,18.615310,19.154826,24.950936,24.075167,22.963721,22.949218,20.168262,23.756601,24.578018,0.003419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,Saale-Orla-Kreis,DEG0K,TH,84.847000,5.540099,1955.848,23051.0,48013.0,NaN,5.593267,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.055401
16076,2012,"Greiz, Landkreis",DEG0L,TH,103.880997,2.040082,1854.354,17851.0,46776.0,NaN,2.091659,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,0.020401
16077,2012,"Altenburger Land, Landkreis",DEG0M,TH,95.289001,-1.616072,1689.681,17732.0,45168.0,NaN,-1.566608,...,NaN,NaN,23.172179,22.076258,21.326727,21.618260,19.454235,21.870640,22.820639,-0.016161


In [9]:
data_check = df_fe[['emp_agr_weight','emp_man_weight','emp_ser_weight','gdp_level','gdp_per_inhabitant','gdp_per_hour_per_employed']]
data_check

,,emp_agr_weight,emp_man_weight,emp_ser_weight,gdp_level,gdp_per_inhabitant,gdp_per_hour_per_employed
Regional_key,year,,,,,,
08111,2004,0.002413,0.213166,0.784421,40680.228,70877.0,58.340000
08115,2004,0.004872,0.422414,0.572714,14478.332,39520.0,47.110001
08116,2004,0.006507,0.376005,0.617488,15205.272,30190.0,42.700001
08117,2004,0.013096,0.400742,0.586162,6158.588,24134.0,38.419998
08118,2004,0.014722,0.341771,0.643507,15851.325,31422.0,47.139999
...,...,...,...,...,...,...,...
16075,2012,0.044653,0.428515,0.526831,1955.848,23051.0,NaN
16076,2012,0.037358,0.373862,0.588780,1854.354,17851.0,NaN
16077,2012,0.028015,0.338902,0.633083,1689.681,17732.0,NaN


<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gdp


##  gdp - no



In [10]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))



                                               Model Comparison                                              
                                     (1)              (2)                       (3)                       (4)
-------------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                       PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                    3573             3573                      3573                      3573
Cov. Est.                      Clustered        Clustered                 Clustered                 Clustered
R-squared                         0.8608           0.4380                    0.3860                    0.4277
R-Squared (Within)                0.1567           0.2604                    0.3005                    0.2615
R-Squared 

In [11]:
# gdp growth

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                  
                                   (9)           (10)
-----------------------------------------------------
Dep. Variable               gdp_growth     gdp_growth
Estimator                     PanelOLS       PanelOLS
No. Observations                  3573           3573
Cov. Est.                    Clustered      Clustered
R-squared                       0.0389         0.1380
R-Squared (Within)              0.0372         0.1908
R-Squared (Between)             0.0703        -78.310
R-Squared (Overall)             0.0389        -3.8758
F-statistic                     20.603         101.31
P-value (F-stat)                0.0000         0.0000
=====================     ============   ============
const                        -0.0596**     -1.5551***
                              (0.0248)       (0.5283)
affected_after_no               0.0053         0.0002
                              (0.0037)       (0.0027)
year_post                   

##  gdp - toas

In [12]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_inhabitant,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_employee,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gdp_per_hour_per_employed,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                                               Model Comparison                                               
                                      (1)              (2)                       (3)                       (4)
--------------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gdp_level     ln_gdp_level     ln_gdp_per_inhabitant     ln_gdp_per_inhabitant
Estimator                        PanelOLS         PanelOLS                  PanelOLS                  PanelOLS
No. Observations                     3573             3573                      3573                      3573
Cov. Est.                       Clustered        Clustered                 Clustered                 Clustered
R-squared                          0.8610           0.4379                    0.3873                    0.4293
R-Squared (Within)                 0.1590           0.2517                    0.3011                    0.2391
R

In [13]:
# gdp growth

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.gdp_growth,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10}, stars = True, precision = "std_errors"))

                   Model Comparison                   
                                    (9)           (10)
------------------------------------------------------
Dep. Variable                gdp_growth     gdp_growth
Estimator                      PanelOLS       PanelOLS
No. Observations                   3573           3573
Cov. Est.                     Clustered      Clustered
R-squared                        0.0383         0.1382
R-Squared (Within)               0.0368         0.1919
R-Squared (Between)              0.0672        -82.803
R-Squared (Overall)              0.0383        -4.1076
F-statistic                      20.310         101.42
P-value (F-stat)                 0.0000         0.0000
======================     ============   ============
const                         -0.0695**     -1.6197***
                               (0.0277)       (0.5297)
affected_after_toas            6.46e-05        -0.0017
                               (0.0031)       (0.0024)
year_post 

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# gva

##  gva - no



In [14]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2,"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))




                                    Model Comparison                                   
                                     (1)              (2)            (3)            (4)
---------------------------------------------------------------------------------------
Dep. Variable               ln_gva_level     ln_gva_level     ln_gva_agr     ln_gva_agr
Estimator                       PanelOLS         PanelOLS       PanelOLS       PanelOLS
No. Observations                    3573             3573           3573           3573
Cov. Est.                      Clustered        Clustered      Clustered      Clustered
R-squared                         0.8609           0.4380         0.6486         0.2640
R-Squared (Within)                0.1201           0.2753         0.2237         0.4090
R-Squared (Between)               0.8676           0.5942         0.6579         0.1855
R-Squared (Overall)               0.8609           0.5913         0.6486         0.1903
F-statistic                     

In [15]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no","gva_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no',"gva_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


# gva - no -significant gva level agr  man fin  trade 

                                            Model Comparison                                           
                                       (9)               (10)                 (11)                 (12)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3573               3573                 3573                 3573
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8291             0.0319               0.7799               0.1623
R-Squared (Within)                  0.1812            -0.0464              -0.0975              -0.1742
R-Squared (Between)                 0.8355             0.4103   

##  gva - toas

In [16]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","gva_man_weight","ln_res","ln_fixed_total"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_level,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_agr_weight","ln_res","ln_fixed_agr"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_man_weight","ln_res","ln_fixed_man"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)


exog_vars = ['affected_after_toas','year_post',"county_affected_toas","gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"gva_ser_weight","ln_res","ln_fixed_ser"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2,"(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


print(compare({"(7)": res_fe_7, "(8)": res_fe_8}, stars = True, precision = "std_errors"))





                                    Model Comparison                                    
                                      (1)              (2)            (3)            (4)
----------------------------------------------------------------------------------------
Dep. Variable                ln_gva_level     ln_gva_level     ln_gva_agr     ln_gva_agr
Estimator                        PanelOLS         PanelOLS       PanelOLS       PanelOLS
No. Observations                     3573             3573           3573           3573
Cov. Est.                       Clustered        Clustered      Clustered      Clustered
R-squared                          0.8611           0.4379         0.6471         0.2636
R-Squared (Within)                 0.1223           0.2665         0.2306         0.4106
R-Squared (Between)                0.8678           0.5984         0.6562         0.1824
R-Squared (Overall)                0.8611           0.5955         0.6471         0.1872
F-statistic          

In [17]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=False)
res_fe_9 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_fin"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_10 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=False)
res_fe_11 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_trade"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_12 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=False)
res_fe_13 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas',"emp_ser_weight","ln_res","ln_fixed_ser_public"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_gva_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_14 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(9)": res_fe_9, "(10)": res_fe_10, "(11)": res_fe_11, "(12)": res_fe_12}, stars = True, precision = "std_errors"))

print(compare({"(13)": res_fe_13, "(14)": res_fe_14}, stars = True, precision = "std_errors"))


#  gva - toas  -significant  man ser  fin  trade  pubilc 

                                            Model Comparison                                            
                                        (9)               (10)                 (11)                 (12)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_gva_ser_fin     ln_gva_ser_fin     ln_gva_ser_trade     ln_gva_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3591               3591                 3591                 3591
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8535             0.0308               0.8199               0.1787
R-Squared (Within)                   0.3643             0.0344               0.2187               0.0099
R-Squared (Between)                  0.8584            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# employee

## employee no

In [18]:
df_fe[['employee', 'emp_agr', 'emp_man','emp_ser', 'emp_ser_fin', 'emp_ser_trade', 'emp_ser_public',
       'ln_emp', 'ln_emp_ser_fin','ln_emp_agr', 'ln_emp_man', 'ln_emp_ser',
       'ln_emp_ser_trade', 'ln_emp_ser_public']]

,,employee,emp_agr,emp_man,emp_ser,emp_ser_fin,emp_ser_trade,emp_ser_public,ln_emp,ln_emp_ser_fin,ln_emp_agr,ln_emp_man,ln_emp_ser,ln_emp_ser_trade,ln_emp_ser_public
Regional_key,year,,,,,,,,,,,,,,
08111,2004,474.066986,1.144,101.055,371.868,127.722,106.565,137.581,13.069104,11.757611,7.042286,11.523420,12.826294,11.576510,11.831968
08115,2004,213.466003,1.040,90.171,122.255,32.859,52.878,36.518,12.271233,10.399981,6.946976,11.409463,11.713864,10.875743,10.505561
08116,2004,248.658997,1.618,93.497,153.544,34.541,66.856,52.147,12.423838,10.449902,7.388946,11.445685,11.941742,11.110296,10.861822
08117,2004,112.932999,1.479,45.257,66.197,12.366,25.701,28.130,11.634550,9.422706,7.299121,10.720113,11.100390,10.154285,10.244592
08118,2004,235.968002,3.474,80.647,151.847,40.387,58.357,53.103,12.371451,10.606263,8.153062,11.297837,11.930629,10.974335,10.879989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,40.736000,1.819,17.456,21.461,4.408,6.981,10.072,10.614867,8.391176,7.506042,9.767439,9.973993,8.850947,9.217515
16076,2012,39.643002,1.481,14.821,23.341,3.864,9.603,9.874,10.587669,8.259458,7.300473,9.603800,10.057967,9.169831,9.197660
16077,2012,37.409000,1.048,12.678,23.683,3.859,9.211,10.613,10.529667,8.258163,6.954639,9.447623,10.072513,9.128154,9.269835


In [19]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_agr_weight","emp_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_agr_weight","emp_man_weight", ]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_agr_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3591           3591           3591           3591
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.9017         0.3192         0.7569         0.2021
R-Squared (Within)              0.3776        -0.1122        -0.2390         0.3092
R-Squared (Between)             0.9027         0.8128         0.7618         0.4094
R-Squared (Overall)             0.9017         0.8111         0.7569         0.4089
F-statistic                     5481.0         372.70         2231.9        

In [20]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3591           3591           3591           3591
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8830         0.6452         0.8941         0.4861
R-Squared (Within)              0.4734         0.3170         0.5236         0.2007
R-Squared (Between)             0.8843         0.6944         0.8950         0.8112
R-Squared (Overall)             0.8830         0.6932         0.8941         0.8098
F-statistic                     5409.4         1928.3         6055.3        

In [21]:
exog_vars = ['affected_after_no','year_post',"county_affected_no","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))

# empl no significant: man and ser (fin trade public)


                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3591               3591                 3591                 3591
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8174             0.1177               0.8812               0.1728
R-Squared (Within)                  0.3855             0.1348               0.1162               0.0743
R-Squared (Between)                 0.8214             0.4986   

## employee toas

In [22]:
df_fe.columns

Index(['Gebietseinheit', 'EU_Code', 'Land', 'resident', 'gdp_growth_100',
       'gdp_level', 'gdp_per_inhabitant', 'gdp_per_employee',
       'gdp_per_hour_per_employed', 'ln_gva', 'gva_level', 'gva_agr',
       'gva_man', 'gva_ser', 'gva_ser_fin', 'gva_ser_trade', 'gva_ser_public',
       'employee', 'emp_agr', 'emp_man', 'emp_ser', 'emp_ser_fin',
       'emp_ser_trade', 'emp_ser_public', 'volume_of_work', 'vow_agr',
       'vow_man', 'vow_ser', 'vow_ser_fin', 'vow_ser_trade', 'vow_ser_public',
       'fixed_agr', 'fixed_man', 'fixed_ser', 'fixed_ser_trade',
       'fixed_ser_fin', 'fixed_ser_public', 'fixed_total', 'firms_no',
       'firms_aff_no', 'No_ratio', 'toas_sum_county', 'toas_aff_sum_county',
       'toas_ratio', 'emp_agr_weight', 'emp_man_weight', 'emp_ser_weight',
       'gva_agr_weight', 'gva_man_weight', 'gva_ser_weight', 'vow_agr_weight',
       'vow_man_weight', 'vow_ser_weight', 'county_affected_no',
       'county_affected_toas', 'year_post', 'affected_after_no',
 

In [23]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public
Regional_key,year,,,,,,,
08111,2004,697.348999,1.935,156.006,539.408,187.321,161.171,190.916
08115,2004,307.300995,2.102,137.127,168.071,44.687,77.277,46.107
08116,2004,356.113007,2.966,141.381,211.767,47.858,97.342,66.568
08117,2004,160.294006,3.153,68.098,89.043,17.513,35.963,35.568
08118,2004,336.239014,6.383,121.486,208.370,55.567,85.251,67.552
...,...,...,...,...,...,...,...,...
16075,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_agr_weight","emp_man_weight","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight","emp_man_weight","emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))





                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_emp         ln_emp     ln_emp_agr     ln_emp_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3591           3591           3591           3591
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.9046         0.3229         0.7572         0.2091
R-Squared (Within)              0.3810        -0.1690        -0.2564         0.3264
R-Squared (Between)             0.9055         0.8198         0.7622         0.4294
R-Squared (Overall)             0.9046         0.8181         0.7572         0.4289
F-statistic                     4853.9         303.18         2235.5        

In [25]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_man_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))

                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_emp_man     ln_emp_man     ln_emp_ser     ln_emp_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3591           3591           3591           3591
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8900         0.6446         0.9003         0.4862
R-Squared (Within)               0.4802         0.2704         0.5337         0.1548
R-Squared (Between)              0.8913         0.7088         0.9011         0.8176
R-Squared (Overall)              0.8900         0.7074         0.9003         0.8160
F-statistic                      5803.5         1923.2         64

In [26]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas","emp_ser_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"emp_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_emp_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))


# empl toas significant total empl agr  man ser (fin public )




                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_emp_ser_fin     ln_emp_ser_fin     ln_emp_ser_trade     ln_emp_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3591               3591                 3591                 3591
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8243             0.1255               0.8902               0.1716
R-Squared (Within)                   0.3973             0.0842               0.1245               0.0527
R-Squared (Between)                  0.8283            

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# volume_of_work

## volume_of_work no of firms


In [27]:
df_fe[['volume_of_work',
       'vow_agr', 'vow_man', 'vow_ser', 'vow_ser_fin','vow_ser_trade', 'vow_ser_public',
       'ln_vow', 
       'ln_vow_ser_fin', 'ln_vow_ser_trade','ln_vow_ser_public', 'ln_vow_agr', 'ln_vow_man', 'ln_vow_ser'
      ]]

,,volume_of_work,vow_agr,vow_man,vow_ser,vow_ser_fin,vow_ser_trade,vow_ser_public,ln_vow,ln_vow_ser_fin,ln_vow_ser_trade,ln_vow_ser_public,ln_vow_agr,ln_vow_man,ln_vow_ser
Regional_key,year,,,,,,,,,,,,,,
08111,2004,697.348999,1.935,156.006,539.408,187.321,161.171,190.916,20.362797,19.048334,18.897976,19.067344,14.475618,18.865405,20.105983
08115,2004,307.300995,2.102,137.127,168.071,44.687,77.277,46.107,19.543338,17.615193,18.162907,17.646475,14.558400,18.736418,18.939897
08116,2004,356.113007,2.966,141.381,211.767,47.858,97.342,66.568,19.690759,17.683749,18.393741,18.013735,14.902725,18.766969,19.170997
08117,2004,160.294006,3.153,68.098,89.043,17.513,35.963,35.568,18.892520,16.678454,17.398001,17.386957,14.963865,18.036458,18.304630
08118,2004,336.239014,6.383,121.486,208.370,55.567,85.251,67.552,19.633333,17.833100,18.261110,18.028408,15.669149,18.615310,19.154826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16075,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16076,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16077,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# data[]"vow_agr_weight","vow_man_weight","vow_ser_weight"

In [29]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_agr_weight","vow_man_weight",'ln_res']
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_agr_weight","vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable                   ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3402           3402           3402           3402
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8928         0.3102         0.7431         0.1978
R-Squared (Within)              0.1658        -0.0463        -0.3140         0.3373
R-Squared (Between)             0.8939         0.8320         0.7489         0.4103
R-Squared (Overall)             0.8928         0.8307         0.7431         0.4099
F-statistic                     4711.6         338.60         1964.4        

In [30]:
exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))




                                  Model Comparison                                 
                                   (1)            (2)            (3)            (4)
-----------------------------------------------------------------------------------
Dep. Variable               ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                     PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                  3402           3402           3402           3402
Cov. Est.                    Clustered      Clustered      Clustered      Clustered
R-squared                       0.8808         0.6320         0.8963         0.5050
R-Squared (Within)              0.4606         0.4663         0.4277         0.3366
R-Squared (Between)             0.8823         0.7392         0.8974         0.8406
R-Squared (Overall)             0.8808         0.7382         0.8963         0.8395
F-statistic                     5018.8         1724.7         5871.0        

In [31]:

exog_vars = ['affected_after_no','year_post',"county_affected_no","vow_ser_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_no','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_no','year_post',"county_affected_no",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_no','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





#  vow no significant man ser (trade public )



                                            Model Comparison                                           
                                       (1)                (2)                  (3)                  (4)
-------------------------------------------------------------------------------------------------------
Dep. Variable               ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                         PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                      3402               3402                 3402                 3402
Cov. Est.                        Clustered          Clustered            Clustered            Clustered
R-squared                           0.8250             0.1432               0.8832               0.2024
R-Squared (Within)                  0.3429             0.1887               0.0689               0.1631
R-Squared (Between)                 0.8295             0.6015   

## volume_of_work toas

In [32]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_agr_weight","vow_man_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight","vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_agr_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_agr,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))



                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                    ln_vow         ln_vow     ln_vow_agr     ln_vow_agr
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3402           3402           3402           3402
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8959         0.3154         0.7430         0.2101
R-Squared (Within)               0.1883        -0.0929        -0.3277         0.3804
R-Squared (Between)              0.8970         0.8382         0.7489         0.4188
R-Squared (Overall)              0.8959         0.8367         0.7430         0.4185
F-statistic                      4868.9         346.95         19

In [33]:
exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_man_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_man,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=False)
res_fe_7 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser,exog,entity_effects=True,time_effects=True)
res_fe_8 = mod.fit(cov_type = 'clustered',cluster_entity=False)



print(compare({"(1)": res_fe_5, "(2)": res_fe_6, "(3)": res_fe_7, "(4)": res_fe_8}, stars = True, precision = "std_errors"))





                                  Model Comparison                                  
                                    (1)            (2)            (3)            (4)
------------------------------------------------------------------------------------
Dep. Variable                ln_vow_man     ln_vow_man     ln_vow_ser     ln_vow_ser
Estimator                      PanelOLS       PanelOLS       PanelOLS       PanelOLS
No. Observations                   3402           3402           3402           3402
Cov. Est.                     Clustered      Clustered      Clustered      Clustered
R-squared                        0.8870         0.6320         0.9015         0.5061
R-Squared (Within)               0.4657         0.4356         0.4387         0.3016
R-Squared (Between)              0.8885         0.7544         0.9025         0.8467
R-Squared (Overall)              0.8870         0.7533         0.9015         0.8455
F-statistic                      5330.2         1724.7         62

In [34]:

exog_vars = ['affected_after_toas','year_post',"county_affected_toas","vow_ser_weight",'ln_res',]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=False)
res_fe_1 = mod.fit(cov_type = 'clustered')


exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_fin,exog,entity_effects=True,time_effects=True)
res_fe_2 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=False)
res_fe_3 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight",]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_trade,exog,entity_effects=True,time_effects=True)
res_fe_4 = mod.fit(cov_type = 'clustered',cluster_entity=False)

exog_vars = ['affected_after_toas','year_post',"county_affected_toas",'ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=False)
res_fe_5 = mod.fit(cov_type = 'clustered')

exog_vars = ['affected_after_toas','ln_res',"vow_ser_weight"]
exog = sm.add_constant(df_fe[exog_vars])
mod = PanelOLS(df_fe.ln_vow_ser_public,exog,entity_effects=True,time_effects=True)
res_fe_6 = mod.fit(cov_type = 'clustered',cluster_entity=False)

print(compare({"(1)": res_fe_1, "(2)": res_fe_2, "(3)": res_fe_3, "(4)": res_fe_4}, stars = True, precision = "std_errors"))

print(compare({"(5)": res_fe_5, "(6)": res_fe_6}, stars = True, precision = "std_errors"))





# vow toas signficant total vow agr ser (fin trade public )



                                            Model Comparison                                            
                                        (1)                (2)                  (3)                  (4)
--------------------------------------------------------------------------------------------------------
Dep. Variable                ln_vow_ser_fin     ln_vow_ser_fin     ln_vow_ser_trade     ln_vow_ser_trade
Estimator                          PanelOLS           PanelOLS             PanelOLS             PanelOLS
No. Observations                       3402               3402                 3402                 3402
Cov. Est.                         Clustered          Clustered            Clustered            Clustered
R-squared                            0.8311             0.1475               0.8911               0.2040
R-Squared (Within)                   0.3512             0.1561               0.0822               0.1595
R-Squared (Between)                  0.8356            

In [35]:
# vow and empl by toas better 
# gva soso
# gdp not ok 

In [36]:
print("done")

done
